In [13]:
import sys
sys.path.append("D:/data/Kaggle/Severstal/tta_wrapper-master/")

In [1]:
#

import pandas as pd

train_df = pd.read_csv('D:/data/Kaggle/Severstal/train.csv')
train_df['ImageId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

print(train_df.shape)
train_df.head()

(50272, 5)


,ImageId_ClassId,EncodedPixels,ImageId,ClassId,hasMask
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,0002cc93b.jpg,1,True
1,0002cc93b.jpg_2,NaN,0002cc93b.jpg,2,False
2,0002cc93b.jpg_3,NaN,0002cc93b.jpg,3,False
3,0002cc93b.jpg_4,NaN,0002cc93b.jpg,4,False
4,00031f466.jpg_1,NaN,00031f466.jpg,1,False


In [2]:
import numpy as np

mask_count_df = train_df.groupby('ImageId').agg(np.sum).reset_index()
mask_count_df.sort_values('hasMask', ascending=False, inplace=True)
print(mask_count_df.shape)
mask_count_df.head()

(12568, 2)


,ImageId,hasMask
10803,db4867ee8.jpg,3.0
11776,ef24da2ba.jpg,3.0
6284,7f30b9c64.jpg,2.0
9421,bf0c81db6.jpg,2.0
9615,c314f43f3.jpg,2.0


In [3]:
#test测试集
import pandas

sub_df = pandas.read_csv('D:/data/Kaggle/Severstal/sample_submission.csv')
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
sub_df.head()

,ImageId_ClassId,EncodedPixels,ImageId
0,004f40c73.jpg_1,1 1,004f40c73.jpg
1,004f40c73.jpg_2,1 1,004f40c73.jpg
2,004f40c73.jpg_3,1 1,004f40c73.jpg
3,004f40c73.jpg_4,1 1,004f40c73.jpg
4,006f39c41.jpg_1,1 1,006f39c41.jpg


In [4]:
test_imgs = pandas.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])#只根据图片名称建表
print(len(test_imgs))
test_imgs.head()

1801


,ImageId
0,004f40c73.jpg
1,006f39c41.jpg
2,00b7fb703.jpg
3,00bbcd9af.jpg
4,0108ce457.jpg


In [5]:
non_missing_train_idx = mask_count_df[mask_count_df['hasMask'] > 0]
non_missing_train_idx.head()

,ImageId,hasMask
10803,db4867ee8.jpg,3.0
11776,ef24da2ba.jpg,3.0
6284,7f30b9c64.jpg,2.0
9421,bf0c81db6.jpg,2.0
9615,c314f43f3.jpg,2.0


In [6]:
import cv2
import os

def load_img(code, base, resize=True):
    path = f'{base}/{code}'
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if resize:
        img = cv2.resize(img, (224, 224))
    
    return img

def validate_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [7]:
from tqdm import tqdm

test_path = 'D:/data/Kaggle/Severstal/tmp/test'
validate_path(test_path)

for code in tqdm(test_imgs['ImageId'].values):
    img = load_img(code, base=r'D:\data\Kaggle\Severstal/test_imgs')
    path = code.replace('.jpg', '')
    cv2.imwrite(f'{test_path}/{path}.png', img)

test_imgs['ImageId'] = test_imgs['ImageId'].apply(lambda x: x.replace('.jpg', '.png'))
sub_df['ImageId'] = sub_df['ImageId'].apply(lambda x: x.replace('.jpg', '.png'))

100%|█████████████████████████████████████████████████████████████████████████████| 1801/1801 [00:15<00:00, 118.46it/s]


In [8]:
from keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 64

test_gen = ImageDataGenerator(rescale=1/255.).flow_from_dataframe(
        test_imgs,
        directory=test_path, #目标目录的路径，如果写了要包含在 dataframe 中映射的所有图像
        x_col='ImageId',
        class_mode=None,
        target_size=(256, 256),
        batch_size=BATCH_SIZE,
        shuffle=False #不打乱顺序
    )


Using TensorFlow backend.


Found 1801 images.


In [9]:
from keras.models import Model, load_model

remove_model = load_model('D:/data/Kaggle/Severstal/pre_model.h5')
remove_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 8, 8, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________

In [11]:
test_missing_pred = remove_model.predict_generator(
    test_gen,
    steps=len(test_gen),
    verbose=1
)

#test_imgs['missingCount'] = test_missing_pred.argmax(axis=1) + 1
test_imgs['allMissing'] = test_missing_pred #allMissing列值等于1代表没有给定的缺陷类型，即无缺陷
test_imgs.head()

,ImageId,allMissing
0,004f40c73.png,0.914483
1,006f39c41.png,0.999879
2,00b7fb703.png,0.999982
3,00bbcd9af.png,0.996996
4,0108ce457.png,0.999664


In [19]:
test_imgs['missingCount'].value_counts()

1    1801
Name: missingCount, dtype: int64

In [12]:
filtered_test_imgs = test_imgs[test_imgs['allMissing'] < 0.5]
print(filtered_test_imgs.shape)
filtered_test_imgs.head()

(770, 2)


,ImageId,allMissing
5,0109b68ec.png,0.070452
6,010ec96b4.png,0.002799
8,017bd7ce3.png,0.022317
10,01d49cd47.png,0.002857
11,020ffb2d3.png,0.058497


In [14]:
filtered_mask = sub_df['ImageId'].isin(filtered_test_imgs["ImageId"].values)
filtered_sub_df = sub_df[filtered_mask].copy()

null_sub_df = sub_df[~filtered_mask].copy()
null_sub_df['EncodedPixels'] = null_sub_df['EncodedPixels'].apply(lambda x: ' ')

filtered_sub_df['ImageId'] = filtered_sub_df['ImageId'].apply(
    lambda x: x.replace('.png', '.jpg'))
filtered_test_imgs['ImageId'] = filtered_test_imgs['ImageId'].apply(
    lambda x: x.replace('.png', '.jpg'))

filtered_sub_df.reset_index(drop=True, inplace=True)
filtered_test_imgs.reset_index(drop=True, inplace=True)

print(filtered_sub_df.shape)
print(null_sub_df.shape)

filtered_sub_df.head()

D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


(3080, 3)
(4124, 3)


,ImageId_ClassId,EncodedPixels,ImageId
0,0109b68ec.jpg_1,1 1,0109b68ec.jpg
1,0109b68ec.jpg_2,1 1,0109b68ec.jpg
2,0109b68ec.jpg_3,1 1,0109b68ec.jpg
3,0109b68ec.jpg_4,1 1,0109b68ec.jpg
4,010ec96b4.jpg_1,1 1,010ec96b4.jpg


In [15]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(rle, input_shape):
    width, height = input_shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]
    
    starts = starts - 1

    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        
    return mask.reshape(height, width).T

def build_masks(rles, input_shape):
    depth = len(rles)
    masks = np.zeros((*input_shape, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, input_shape)
    
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i])
            for i in range(depth)]
    
    return rles

print(train_df.loc[0,'EncodedPixels'])
mask_tensor = rle2mask(train_df.loc[0,'EncodedPixels'], (256,1600,1))
print(mask_tensor, mask_tensor.shape)
print(mask2rle(rle2mask(train_df.loc[0,'EncodedPixels'], (256,1600,1))))
mask2rle(rle2mask('1 1',((256,1600,1))))

29102 12 29346 24 29602 24 29858 24 30114 24 30370 24 30626 24 30882 24 31139 23 31395 23 31651 23 31907 23 32163 23 32419 23 32675 23 77918 27 78174 55 78429 60 78685 64 78941 68 79197 72 79452 77 79708 81 79964 85 80220 89 80475 94 80731 98 80987 102 81242 105 81498 105 81754 104 82010 104 82265 105 82521 31 82556 69 82779 27 82818 63 83038 22 83080 57 83297 17 83342 50 83555 13 83604 44 83814 8 83866 37 84073 3 84128 31 84390 25 84652 18 84918 8 85239 10 85476 29 85714 47 85960 57 86216 57 86471 58 86727 58 86983 58 87238 59 87494 59 87750 59 88005 60 88261 60 88517 60 88772 61 89028 53 89283 40 89539 32 89667 10 89795 30 89923 28 90050 29 90179 37 90306 27 90434 38 90562 14 90690 38 90817 9 90946 38 91073 3 91202 38 91458 38 91714 38 91969 39 92225 39 92481 39 92737 39 92993 39 93248 40 93504 40 93760 40 94026 30 94302 10 189792 7 190034 21 190283 28 190539 28 190795 28 191051 28 191307 28 191563 28 191819 28 192075 28 192331 28 192587 28 192843 23 193099 14 193355 5
[[0 0 0 ... 0 

'1 1'

In [40]:
from skimage.color import gray2rgb
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path='D:/data/Kaggle/Severstal/train_imgs',
                 batch_size=32, dim=(256, 1600), n_channels=3,
                 n_classes=4, random_state=2019, shuffle=True,aug=False):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        self.seq = aug
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
                
        X = self.__generate_X(list_IDs_batch)
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            return X, y.astype(int)
        
        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, list_IDs_batch):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            img_path = f"{self.base_path}/{im_name}"
            img = self.__load_grayscale(img_path)
            
            # Store samples
            X[i,] = gray2rgb(img[:,:,0])

        return X
    
    def __generate_y(self, list_IDs_batch):
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            image_df = self.target_df[self.target_df['ImageId'] == im_name]
            
            rles = image_df['EncodedPixels'].values
            masks = build_masks(rles, input_shape=self.dim)
            
            y[i, ] = masks
        return y
    
    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)

        return img
    
    def __load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img

In [41]:
BATCH_SIZE = 8
import imgaug as ia
import imgaug.augmenters as iaa

from keras.models import Sequential
from sklearn.model_selection import train_test_split

seq = iaa.Sequential([
    iaa.Sharpen((0.0, 1.0)),       # sharpen the image
    iaa.Fliplr(),
    iaa.Flipud(),
    iaa.ElasticTransformation(alpha=50, sigma=5)  # apply water effect (affects segmaps)
], random_order=True)


train_idx, val_idx = train_test_split(
    non_missing_train_idx.index,  # NOTICE DIFFERENCE
    random_state=444, 
    test_size=0.15
)

train_generator = DataGenerator(
    train_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4,
    aug = seq
)

val_generator = DataGenerator(
    val_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4,
    aug = seq
)

In [30]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

i=0
plt.imshow(y[i].sum(axis=-1))


Bad key "lines.markerfacecolor" on line 11 in
D:\Selvaria\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution

Bad key "lines.markeredgecolor" on line 12 in
D:\Selvaria\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


KeyError: 'keymap.help'

In [21]:
from keras.losses import binary_crossentropy

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import  ModelCheckpoint
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization
from tqdm import tqdm_notebook
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.engine import InputSpec
from keras import backend as K
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.losses import binary_crossentropy
import keras.callbacks as callbacks
from keras.callbacks import Callback
from keras.applications.xception import Xception
from keras.layers import multiply


from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects

from keras.engine.topology import Input
from keras.engine.training import Model
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers.core import Activation, SpatialDropout2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.regularizers import l2
from keras.layers.core import Dense, Lambda
from keras.layers.merge import concatenate, add
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, Permute
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation == True:
        x = LeakyReLU(alpha=0.1)(x)
    return x

def residual_block(blockInput, num_filters=16):
    x = LeakyReLU(alpha=0.1)(blockInput)
    x = BatchNormalization()(x)
    blockInput = BatchNormalization()(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    return x

In [36]:
import efficientnet as efn 

def UEfficientNet(input_shape=(None, None, 3),dropout_rate=0.1):

    backbone = efn.EfficientNetB2(weights=None,
                            include_top=False,
                            input_shape=input_shape)
#     backbone.load_weights("../input/efficientnet-keras-weights-b0b5/efficientnet-b2_imagenet_1000_notop.h5")
    inputs = backbone.input
    start_neurons = 8
    
    i=2
    lr = []
    for l in backbone.layers:
        if l.name == 'block{}a_expand_activation'.format(i):
            lr.append(l)
            i+=1

    print(len(lr))
    conv4 = lr[-1].output
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(dropout_rate)(pool4)
    
     # Middle
    convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 32)
    convm = residual_block(convm,start_neurons * 32)
    convm = LeakyReLU(alpha=0.1)(convm)
    
    deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
    deconv4_up1 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(deconv4)
    deconv4_up2 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(deconv4_up1)
    deconv4_up3 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(deconv4_up2)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(dropout_rate)(uconv4) 
    
    uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 16)
#     uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = LeakyReLU(alpha=0.1)(uconv4)  #conv1_2
    
    deconv3 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv4)
    deconv3_up1 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(deconv3)
    deconv3_up2 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(deconv3_up1)
    conv3 = lr[-2].output
    uconv3 = concatenate([deconv3,deconv4_up1, conv3])    
    uconv3 = Dropout(dropout_rate)(uconv3)
    
    uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 8)
#     uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = LeakyReLU(alpha=0.1)(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv3)
    deconv2_up1 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(deconv2)
    conv2 = lr[-4].output
    uconv2 = concatenate([deconv2,deconv3_up1,deconv4_up2, conv2])
        
    uconv2 = Dropout(0.1)(uconv2)
    uconv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 4)
#     uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = LeakyReLU(alpha=0.1)(uconv2)
    
    deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
    conv1 = lr[-5].output
    uconv1 = concatenate([deconv1,deconv2_up1,deconv3_up2,deconv4_up3, conv1])
    
    uconv1 = Dropout(0.1)(uconv1)
    uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 2)
#     uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = LeakyReLU(alpha=0.1)(uconv1)
    
    uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
    uconv0 = Dropout(0.1)(uconv0)
    uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
    uconv0 = residual_block(uconv0,start_neurons * 1)
#     uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = LeakyReLU(alpha=0.1)(uconv0)
    
    uconv0 = Dropout(dropout_rate/2)(uconv0)
    uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv0) 
    output_layer = Conv2D(4, (1,1), padding="same", activation="sigmoid")(uconv0)    
    
    model = Model(inputs, output_layer)
    model.name = 'u-xception'

    return model

In [37]:
model = UEfficientNet(input_shape=(256,1600,3))
model.summary()

0


IndexError: list index out of range

In [27]:
from keras import backend as K
from keras.losses import binary_crossentropy

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def bce_logdice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - K.log(1. - dice_loss(y_true, y_pred))

In [43]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate

inputs = Input((256, 1600, 3)) #原图尺寸，黑白单通道

c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (inputs)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(64, (3, 3), activation='relu', padding='same') (p4)
c5 = Conv2D(64, (3, 3), activation='relu', padding='same') (c5)
p5 = MaxPooling2D(pool_size=(2, 2)) (c5) #到这里一直是在对原始的input进行层操作变换

c55 = Conv2D(128, (3, 3), activation='relu', padding='same') (p5) #开始对原始的input进行变换，直到u6的反向卷积
c55 = Conv2D(128, (3, 3), activation='relu', padding='same') (c55)

#应该是使用了残差连接，用于避免梯度消失和表示瓶颈
u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c55) #转置卷积层 (有时被成为反卷积)，将具有该卷积层输出shape的tensor转换为具有该卷积层输入shape的tensor。同时保留与卷积层兼容的连接模式
u6 = concatenate([u6, c5]) #和上一步在一起，相当于降维后连结在一起，和之前的维度相同。默认axis=-1(最后一个轴)，相当于按照c5[-1轴],u6[-1轴]拼接，即
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u71 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u71 = concatenate([u71, c4])
c71 = Conv2D(32, (3, 3), activation='relu', padding='same') (u71)
c61 = Conv2D(32, (3, 3), activation='relu', padding='same') (c71)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c61)
u7 = concatenate([u7, c3])
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

outputs = Conv2D(4, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_coef])#使用之前写的评分函数
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 256, 1600, 3) 0                                            
__________________________________________________________________________________________________
conv2d_507 (Conv2D)             (None, 256, 1600, 8) 224         input_8[0][0]                    
__________________________________________________________________________________________________
conv2d_508 (Conv2D)             (None, 256, 1600, 8) 584         conv2d_507[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_11 (MaxPooling2D) (None, 128, 800, 8)  0           conv2d_508[0][0]                 
__________________________________________________________________________________________________
conv2d_509

conv2d_528 (Conv2D)             (None, 256, 1600, 8) 584         conv2d_527[0][0]                 
__________________________________________________________________________________________________
conv2d_529 (Conv2D)             (None, 256, 1600, 4) 36          conv2d_528[0][0]                 
Total params: 600,756
Trainable params: 600,756
Non-trainable params: 0
__________________________________________________________________________________________________


In [ ]:
from keras.callbacks import Callback, ModelCheckpoint

checkpoint = ModelCheckpoint(
    'D:/data/Kaggle/Severstal/model_2.1.h5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

history = model.fit_generator(
    train_generator,
    validation_data=val_generator,
    callbacks=[checkpoint],
    use_multiprocessing=False,
    workers=1,
    epochs=10
)

Epoch 1/10


169/708 [======>.......................] - ETA: 5:26:31 - loss: 1.6773 - dice_coef: 7.5411e- - ETA: 3:24:29 - loss: 1.6821 - dice_coef: 3.8764e- - ETA: 2:43:31 - loss: 1.6818 - dice_coef: 2.6343e- - ETA: 2:23:20 - loss: 1.6805 - dice_coef: 0.0011   - ETA: 2:10:35 - loss: 1.6792 - dice_coef: 9.1534e- - ETA: 2:02:10 - loss: 1.6752 - dice_coef: 7.6279e- - ETA: 1:56:13 - loss: 1.6716 - dice_coef: 0.0013   - ETA: 1:52:04 - loss: 1.6687 - dice_coef: 0.00 - ETA: 1:48:36 - loss: 1.6635 - dice_coef: 0.00 - ETA: 1:45:43 - loss: 1.6591 - dice_coef: 9.0536e- - ETA: 1:43:23 - loss: 1.6517 - dice_coef: 8.2305e- - ETA: 1:41:27 - loss: 1.6451 - dice_coef: 0.0017   - ETA: 1:39:43 - loss: 1.6362 - dice_coef: 0.00 - ETA: 1:38:20 - loss: 1.6351 - dice_coef: 0.00 - ETA: 1:37:23 - loss: 1.6296 - dice_coef: 0.00 - ETA: 1:36:12 - loss: 1.6233 - dice_coef: 0.00 - ETA: 1:35:16 - loss: 1.6170 - dice_coef: 0.00 - ETA: 1:34:27 - loss: 1.6120 - dice_coef: 0.00 - ETA: 1:33:32 - loss: 1.6070 - dice_coef: 0.00 - ETA: 

345/708 [=============>................] - ETA: 1:06:53 - loss: 1.1436 - dice_coef: 0.00 - ETA: 1:06:44 - loss: 1.1428 - dice_coef: 0.00 - ETA: 1:06:35 - loss: 1.1420 - dice_coef: 0.00 - ETA: 1:06:26 - loss: 1.1412 - dice_coef: 0.00 - ETA: 1:06:17 - loss: 1.1405 - dice_coef: 0.00 - ETA: 1:06:08 - loss: 1.1401 - dice_coef: 0.00 - ETA: 1:05:59 - loss: 1.1392 - dice_coef: 0.00 - ETA: 1:05:50 - loss: 1.1388 - dice_coef: 0.00 - ETA: 1:05:42 - loss: 1.1379 - dice_coef: 0.00 - ETA: 1:05:33 - loss: 1.1372 - dice_coef: 0.00 - ETA: 1:05:24 - loss: 1.1365 - dice_coef: 0.00 - ETA: 1:05:15 - loss: 1.1358 - dice_coef: 0.00 - ETA: 1:05:06 - loss: 1.1350 - dice_coef: 0.00 - ETA: 1:04:58 - loss: 1.1342 - dice_coef: 0.00 - ETA: 1:04:49 - loss: 1.1334 - dice_coef: 0.00 - ETA: 1:04:41 - loss: 1.1327 - dice_coef: 0.00 - ETA: 1:04:32 - loss: 1.1320 - dice_coef: 0.00 - ETA: 1:04:23 - loss: 1.1312 - dice_coef: 0.00 - ETA: 1:04:14 - loss: 1.1308 - dice_coef: 0.00 - ETA: 1:04:06 - loss: 1.1302 - dice_coef: 0.00

523/708 [=====================>........] - ETA: 43:28 - loss: 1.0643 - dice_coef: 0.01 - ETA: 43:21 - loss: 1.0641 - dice_coef: 0.01 - ETA: 43:14 - loss: 1.0637 - dice_coef: 0.01 - ETA: 43:06 - loss: 1.0633 - dice_coef: 0.01 - ETA: 42:59 - loss: 1.0630 - dice_coef: 0.01 - ETA: 42:51 - loss: 1.0626 - dice_coef: 0.01 - ETA: 42:44 - loss: 1.0622 - dice_coef: 0.02 - ETA: 42:36 - loss: 1.0620 - dice_coef: 0.02 - ETA: 42:29 - loss: 1.0618 - dice_coef: 0.02 - ETA: 42:22 - loss: 1.0607 - dice_coef: 0.02 - ETA: 42:14 - loss: 1.0602 - dice_coef: 0.02 - ETA: 42:07 - loss: 1.0593 - dice_coef: 0.02 - ETA: 42:00 - loss: 1.0592 - dice_coef: 0.02 - ETA: 41:52 - loss: 1.0588 - dice_coef: 0.02 - ETA: 41:45 - loss: 1.0595 - dice_coef: 0.02 - ETA: 41:37 - loss: 1.0594 - dice_coef: 0.02 - ETA: 41:30 - loss: 1.0590 - dice_coef: 0.02 - ETA: 41:23 - loss: 1.0588 - dice_coef: 0.02 - ETA: 41:15 - loss: 1.0586 - dice_coef: 0.02 - ETA: 41:08 - loss: 1.0586 - dice_coef: 0.02 - ETA: 41:00 - loss: 1.0585 - dice_coef

701/708 [============================>.] - ETA: 21:53 - loss: 1.0278 - dice_coef: 0.05 - ETA: 21:46 - loss: 1.0277 - dice_coef: 0.05 - ETA: 21:39 - loss: 1.0275 - dice_coef: 0.05 - ETA: 21:32 - loss: 1.0275 - dice_coef: 0.05 - ETA: 21:25 - loss: 1.0271 - dice_coef: 0.05 - ETA: 21:18 - loss: 1.0270 - dice_coef: 0.05 - ETA: 21:10 - loss: 1.0268 - dice_coef: 0.05 - ETA: 21:03 - loss: 1.0269 - dice_coef: 0.05 - ETA: 20:56 - loss: 1.0266 - dice_coef: 0.05 - ETA: 20:49 - loss: 1.0266 - dice_coef: 0.05 - ETA: 20:42 - loss: 1.0266 - dice_coef: 0.05 - ETA: 20:34 - loss: 1.0263 - dice_coef: 0.05 - ETA: 20:27 - loss: 1.0263 - dice_coef: 0.05 - ETA: 20:20 - loss: 1.0263 - dice_coef: 0.05 - ETA: 20:13 - loss: 1.0263 - dice_coef: 0.05 - ETA: 20:06 - loss: 1.0263 - dice_coef: 0.05 - ETA: 19:58 - loss: 1.0263 - dice_coef: 0.05 - ETA: 19:51 - loss: 1.0260 - dice_coef: 0.05 - ETA: 19:44 - loss: 1.0259 - dice_coef: 0.05 - ETA: 19:37 - loss: 1.0258 - dice_coef: 0.05 - ETA: 19:30 - loss: 1.0257 - dice_coef

...

708/708 [==============================] - ETA: 46s - loss: 0.7717 - dice_coef: 0.29 - ETA: 38s - loss: 0.7719 - dice_coef: 0.29 - ETA: 30s - loss: 0.7721 - dice_coef: 0.29 - ETA: 23s - loss: 0.7725 - dice_coef: 0.29 - ETA: 15s - loss: 0.7722 - dice_coef: 0.29 - ETA: 7s - loss: 0.7725 - dice_coef: 0.2979 - 5842s 8s/step - loss: 0.7727 - dice_coef: 0.2978 - val_loss: 0.7448 - val_dice_coef: 0.3192
Epoch 3/10


170/708 [======>.......................] - ETA: 1:34:16 - loss: 0.6507 - dice_coef: 0.42 - ETA: 1:31:03 - loss: 0.7863 - dice_coef: 0.28 - ETA: 1:30:00 - loss: 0.7706 - dice_coef: 0.30 - ETA: 1:28:49 - loss: 0.7821 - dice_coef: 0.30 - ETA: 1:28:03 - loss: 0.7636 - dice_coef: 0.31 - ETA: 1:27:55 - loss: 0.7884 - dice_coef: 0.28 - ETA: 1:27:30 - loss: 0.8243 - dice_coef: 0.24 - ETA: 1:27:22 - loss: 0.8173 - dice_coef: 0.25 - ETA: 1:27:46 - loss: 0.8271 - dice_coef: 0.24 - ETA: 1:28:04 - loss: 0.8130 - dice_coef: 0.25 - ETA: 1:27:56 - loss: 0.8034 - dice_coef: 0.26 - ETA: 1:27:36 - loss: 0.7903 - dice_coef: 0.27 - ETA: 1:27:39 - loss: 0.7824 - dice_coef: 0.28 - ETA: 1:27:38 - loss: 0.7879 - dice_coef: 0.27 - ETA: 1:27:34 - loss: 0.7887 - dice_coef: 0.27 - ETA: 1:27:20 - loss: 0.8041 - dice_coef: 0.25 - ETA: 1:27:00 - loss: 0.7916 - dice_coef: 0.26 - ETA: 1:26:43 - loss: 0.7754 - dice_coef: 0.28 - ETA: 1:26:38 - loss: 0.7783 - dice_coef: 0.28 - ETA: 1:26:33 - loss: 0.7822 - dice_coef: 0.29

346/708 [=============>................] - ETA: 1:06:06 - loss: 0.6907 - dice_coef: 0.37 - ETA: 1:05:59 - loss: 0.6890 - dice_coef: 0.37 - ETA: 1:05:51 - loss: 0.6879 - dice_coef: 0.37 - ETA: 1:05:44 - loss: 0.6898 - dice_coef: 0.37 - ETA: 1:05:36 - loss: 0.6883 - dice_coef: 0.37 - ETA: 1:05:29 - loss: 0.6891 - dice_coef: 0.37 - ETA: 1:05:21 - loss: 0.6876 - dice_coef: 0.37 - ETA: 1:05:14 - loss: 0.6884 - dice_coef: 0.37 - ETA: 1:05:06 - loss: 0.6872 - dice_coef: 0.37 - ETA: 1:04:58 - loss: 0.6882 - dice_coef: 0.37 - ETA: 1:04:50 - loss: 0.6876 - dice_coef: 0.37 - ETA: 1:04:42 - loss: 0.6894 - dice_coef: 0.37 - ETA: 1:04:34 - loss: 0.6887 - dice_coef: 0.37 - ETA: 1:04:27 - loss: 0.6874 - dice_coef: 0.37 - ETA: 1:04:19 - loss: 0.6865 - dice_coef: 0.37 - ETA: 1:04:11 - loss: 0.6864 - dice_coef: 0.37 - ETA: 1:04:04 - loss: 0.6839 - dice_coef: 0.37 - ETA: 1:03:56 - loss: 0.6849 - dice_coef: 0.37 - ETA: 1:03:49 - loss: 0.6839 - dice_coef: 0.37 - ETA: 1:03:41 - loss: 0.6852 - dice_coef: 0.37

524/708 [=====================>........] - ETA: 44:15 - loss: 0.6960 - dice_coef: 0.36 - ETA: 44:08 - loss: 0.6970 - dice_coef: 0.36 - ETA: 44:01 - loss: 0.6970 - dice_coef: 0.36 - ETA: 43:53 - loss: 0.6961 - dice_coef: 0.36 - ETA: 43:46 - loss: 0.6967 - dice_coef: 0.36 - ETA: 43:39 - loss: 0.6967 - dice_coef: 0.36 - ETA: 43:31 - loss: 0.6964 - dice_coef: 0.36 - ETA: 43:24 - loss: 0.6963 - dice_coef: 0.36 - ETA: 43:16 - loss: 0.6955 - dice_coef: 0.36 - ETA: 43:09 - loss: 0.6948 - dice_coef: 0.36 - ETA: 43:01 - loss: 0.6950 - dice_coef: 0.36 - ETA: 42:54 - loss: 0.6948 - dice_coef: 0.36 - ETA: 42:46 - loss: 0.6949 - dice_coef: 0.36 - ETA: 42:39 - loss: 0.6945 - dice_coef: 0.36 - ETA: 42:31 - loss: 0.6947 - dice_coef: 0.36 - ETA: 42:24 - loss: 0.6939 - dice_coef: 0.36 - ETA: 42:17 - loss: 0.6942 - dice_coef: 0.36 - ETA: 42:10 - loss: 0.6950 - dice_coef: 0.36 - ETA: 42:02 - loss: 0.6944 - dice_coef: 0.36 - ETA: 41:55 - loss: 0.6942 - dice_coef: 0.36 - ETA: 41:47 - loss: 0.6945 - dice_coef

637/708 [=========================>....] - ETA: 22:26 - loss: 0.6851 - dice_coef: 0.37 - ETA: 22:18 - loss: 0.6845 - dice_coef: 0.37 - ETA: 22:11 - loss: 0.6846 - dice_coef: 0.37 - ETA: 22:04 - loss: 0.6843 - dice_coef: 0.37 - ETA: 21:56 - loss: 0.6840 - dice_coef: 0.37 - ETA: 21:49 - loss: 0.6842 - dice_coef: 0.37 - ETA: 21:42 - loss: 0.6843 - dice_coef: 0.37 - ETA: 21:34 - loss: 0.6846 - dice_coef: 0.37 - ETA: 21:27 - loss: 0.6853 - dice_coef: 0.37 - ETA: 21:20 - loss: 0.6850 - dice_coef: 0.37 - ETA: 21:13 - loss: 0.6848 - dice_coef: 0.37 - ETA: 21:06 - loss: 0.6844 - dice_coef: 0.37 - ETA: 20:58 - loss: 0.6850 - dice_coef: 0.37 - ETA: 20:51 - loss: 0.6845 - dice_coef: 0.37 - ETA: 20:44 - loss: 0.6839 - dice_coef: 0.37 - ETA: 20:37 - loss: 0.6831 - dice_coef: 0.38 - ETA: 20:30 - loss: 0.6833 - dice_coef: 0.38 - ETA: 20:23 - loss: 0.6825 - dice_coef: 0.38 - ETA: 20:16 - loss: 0.6825 - dice_coef: 0.38 - ETA: 20:08 - loss: 0.6825 - dice_coef: 0.38 - ETA: 20:01 - loss: 0.6821 - dice_coef

In [18]:
from tta_wrapper import tta_segmentation

In [30]:
#model.load_weights('D:/data/Kaggle/Severstal/modelB2tta.h5')
#print("load_model")
test_df = []
tta_model = tta_segmentation(model, h_flip=True, h_shift=(-10, 10),mul =[0.95], merge='mean')

for i in range(0, filtered_test_imgs.shape[0], 300):
    batch_idx = list(
        range(i, min(filtered_test_imgs.shape[0], i + 300))
    )
    
    test_generator = DataGenerator(
        batch_idx,
        df=filtered_test_imgs,
        shuffle=False,
        mode='predict',
        base_path='D:/data/Kaggle/Severstal/test_imgs',
        target_df=filtered_sub_df,
        batch_size=1,
        n_classes=4,aug = None
    )
    
    batch_pred_masks = tta_model.predict_generator(
        test_generator, 
        workers=1,
        verbose=1,
        use_multiprocessing=False
    )
    
    for j, b in tqdm(enumerate(batch_idx)):
        filename = filtered_test_imgs['ImageId'].iloc[b]
        image_df = filtered_sub_df[filtered_sub_df['ImageId'] == filename].copy()
        
#         pred_masks = batch_pred_masks[j, ].round().astype(int)
        pred_masks = batch_pred_masks[j, ]
        pred_masks[pred_masks<0.95]=0
        pred_masks[pred_masks>0.5]=1
        pred_rles = build_rles(pred_masks.astype(int))
        
        image_df['EncodedPixels'] = pred_rles
        test_df.append(image_df)
        
    gc.collect()

ValueError: You are trying to load a weight file containing 229 layers into a model with 28 layers.